In [1]:
from web3 import Web3
from solcx import compile_source, install_solc, set_solc_version

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
ADDRESS = "0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3"
w3.eth.default_account = Web3.to_checksum_address(ADDRESS)

print("Using account:", w3.eth.default_account)

set_solc_version("0.8.19")

contract_source = """
pragma solidity ^0.8.19;
contract FeedbackIntegrity {
    struct FeedbackRecord {
        bytes32 feedbackHash;
        uint256 timestamp;
        address submittedBy;
    }

    FeedbackRecord[] public records;

    function storeFeedbackHash(bytes32 _hash) public {
        records.push(FeedbackRecord(_hash, block.timestamp, msg.sender));
    }

    function getCount() public view returns (uint256) {
        return records.length;
    }
}
"""

compiled = compile_source(contract_source)
contract_interface = compiled['<stdin>:FeedbackIntegrity']

Feedback = w3.eth.contract(
    abi=contract_interface['abi'],
    bytecode=contract_interface['bin']
)

tx_hash = Feedback.constructor().transact({
    "from": w3.eth.default_account,
    "gas": 3000000
})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi']
)

print("Contract deployed at:", tx_receipt.contractAddress)

Using account: 0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3
Contract deployed at: 0x5C8D13b3206EEC58b457589D570E4ED3aaEAB34C


In [2]:
import hashlib
import json

# Original feedback
original_feedback = {
    "question_1_rating": 5,
    "question_2_rating": 5,
    "question_3_rating": 5,
    "question_4_rating": 5,
    "question_5_rating": 5,
    "comment": "The modules are very good."
}

In [3]:
# Compute hash
original_hash = hashlib.sha256(json.dumps(original_feedback, sort_keys=True).encode()).hexdigest()

In [4]:
# Submit to blockchain
tx = contract.functions.storeFeedbackHash(bytes.fromhex(original_hash)).transact({
    "from": w3.eth.default_account,
    "gas": 300000
})
w3.eth.wait_for_transaction_receipt(tx)

AttributeDict({'transactionHash': HexBytes('0xa9bdedcd7af1cc6de5bdce13493b111cfe44ac9a2a5a70004c14d1aad35744f3'),
 'transactionIndex': 0,
 'blockNumber': 9,
 'blockHash': HexBytes('0x17490d8a4e360c14225185c9068411e1d8a135d001ffdb2439b8cde349768504'),
 'from': '0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3',
 'to': '0x5C8D13b3206EEC58b457589D570E4ED3aaEAB34C',
 'cumulativeGasUsed': 110744,
 'gasUsed': 110744,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status':

In [5]:
# Get latest record from blockchain
latest_index = contract.functions.getCount().call() - 1
record = contract.functions.records(latest_index).call()
onchain_hash = record[0].hex()

# Verify integrity
if original_hash == onchain_hash:
    print("✅ Feedback valid")
else:
    print("❌ Feedback tampered")

✅ Feedback valid


In [6]:
# Tamper feedback locally
tampered_feedback = original_feedback.copy()
tampered_feedback["question_1_rating"] = 1

# Recompute hash
tampered_hash = hashlib.sha256(json.dumps(tampered_feedback, sort_keys=True).encode()).hexdigest()

In [7]:
# Get latest record from blockchain
latest_index = contract.functions.getCount().call() - 1
record = contract.functions.records(latest_index).call()
onchain_hash = record[0].hex()

# Verify integrity
if tampered_hash == onchain_hash:
    print("✅ Feedback valid")
else:
    print("❌ Feedback tampered")

❌ Feedback tampered


In [8]:
count = contract.functions.getCount().call() 
print("Total module feedback records on-chain:", count)

Total module feedback records on-chain: 1
